# Classification - Acquire Exercises

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pydataset import data
import os
from env import host, user, password
import acquire as a

#### 1.  In a jupyter notebook, classification_exercises.ipynb, use a python module (pydata or seaborn datasets) containing datasets as a source from the iris data. Create a pandas dataframe, df_iris, from this data.

In [ ]:
iris = data("iris")

In [ ]:
#print the first three rows:
iris.head(3)

In [ ]:
#print the number of rows and columns (shape)
iris.shape

In [ ]:
#print the column names:
iris.columns

In [ ]:
#print the data type of each column
iris.dtypes

In [ ]:
#more detailed look
iris.info()

#### print the summary statistics for each of the numeric variables. 
Would you recommend rescaling the data based on these statistics?

In [ ]:
#summary statistics
stats = iris.describe().T
stats

#### I would not recommend rescaling because they are all measured in cm.

### 2. Read the Table1_CustDetails table from the Excel_Exercises.xlsx file into a dataframe named df_excel.

In [ ]:
df_excel = pd.read_excel('Spreadsheets_Exercises.xlsx', sheet_name='Table1_CustDetails')
df_excel.head()

In [ ]:
#assign the first 100 rows to a new df df_excel_sample
df_excel_sample = df_excel.head(100)
df_excel_sample.shape

In [ ]:
#print the number of rows of your original dataframe
df_excel.shape[0]

In [ ]:
#print the first 5 column names
df_excel.columns[:5]

In [ ]:
#print the column names that have a data type of object
object_columns = df_excel.select_dtypes(include='object')
object_columns.columns

In [ ]:
#compute range of numeric values

#create df with just numeric values and describe with stats to show min and max
numeric_df = df_excel[['monthly_charges', 'total_charges']].describe().T

In [ ]:
#use stats and use min and max to add new column with range
numeric_df['range'] = numeric_df['max'] - numeric_df['min']

In [ ]:
#show only range column
numeric_df['range']

### 3.  Read the data from this google sheet into a dataframe, df_google

In [ ]:
#create variable with URL
sheet_url = 'https://docs.google.com/spreadsheets/d/1Uhtml8KY19LILuZsrDtlsHHDC9wuDGUSe8LTEwvdI5g/edit#gid=341089357'

In [ ]:
#export to csv using .replace
csv_export_url = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')

In [ ]:
#create df
df_google = pd.read_csv(csv_export_url)
df_google.head()

In [ ]:
# print first 3 rows
df_google.head(3)

In [ ]:
#print the number of rows and columns
df_google.shape

In [ ]:
df_google.columns.tolist()

In [ ]:
#print the data type of each column
df_google.dtypes

In [ ]:
#print summary statistics
df_google.describe().T

In [ ]:
#print the unique values for each of your categoricals
for column in df_google.select_dtypes(include='object').columns:
    print(df_google[column].value_counts())

## Acquire.py Functions

In [ ]:
def get_connection(db, user=user, host=host, password=password):
    '''
    This function uses my info from my env file to
    create a connection url to access the Codeup db.
    It takes in a string name of a database as an argument.
    '''
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

#### 1.  Make a function named get_titanic_data that returns the titanic data from the codeup data science database as a pandas data frame. Obtain your data from the Codeup Data Science Database.

In [ ]:
def new_titanic_data():
    '''
    This function reads the titanic data from the Codeup db into a df,
    write it to a csv file, and returns the df.
    '''
    # Create SQL query.
    sql_query = 'SELECT * FROM passengers'
    
    # Read in DataFrame from Codeup db.
    df = pd.read_sql(sql_query, get_connection('titanic_db'))
    
    return df

In [ ]:
def get_titanic_data():
    '''
    This function reads in titanic data from Codeup database, writes data to
    a csv file if a local file does not exist, and returns a df.
    '''
    if os.path.isfile('titanic_df.csv'):
        
        # If csv file exists, read in data from csv file.
        df = pd.read_csv('titanic_df.csv', index_col=0)
        
    else:
        
        # Read fresh data from db into a DataFrame.
        df = new_titanic_data()
        
        # Write DataFrame to a csv file.
        df.to_csv('titanic_df.csv')
        
    return df

#### 2.  Make a function named get_iris_data that returns the data from the iris_db on the codeup data science database as a pandas data frame. The returned data frame should include the actual name of the species in addition to the species_ids. Obtain your data from the Codeup Data Science Database.

In [ ]:
def get_iris_data():
    '''
    This function reads in iris data from Codeup database, writes data to
    a csv file if a local file does not exist, and returns a df.
    '''
    if os.path.isfile('iris_df.csv'):
        
        # If csv file exists read in data from csv file.
        df = pd.read_csv('iris_df.csv', index_col=0)
        
    else:
        
        # Read fresh data from db into a DataFrame
        df = new_iris_data()
        
        # Cache data
        df.to_csv('iris_df.csv')
        
    return df

In [ ]:
def new_iris_data():
    '''
    This function reads the iris data from the Codeup db into a df.
    '''
    sql_query = """
                SELECT 
                    species_id,
                    species_name,
                    sepal_length,
                    sepal_width,
                    petal_length,
                    petal_width
                FROM measurements
                JOIN species USING(species_id)
                """
    
    # Read in DataFrame from Codeup db.
    df = pd.read_sql(sql_query, get_connection('iris_db'))
    
    return df

#### 3.  Once you've got your get_titanic_data and get_iris_data functions written, now it's time to add caching to them. To do this, edit the beginning of the function to check for a local filename like titanic.csv or iris.csv. If they exist, use the .csv file. If the file doesn't exist, then produce the SQL and pandas necessary to create a dataframe, then write the dataframe to a .csv file with the appropriate name.

In [ ]:
#run functions in acquire.py to create csv files
a.new_titanic_data().head()

In [ ]:
#create csv
a.get_titanic_data()

#### Iris data from acquire.py

In [ ]:
a.new_iris_data()

In [ ]:
#create csv
a.get_iris_data()

# Classification - Prepare Exercises

In [ ]:
# import splitting and imputing functions
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

# turn off pink boxes for demo
import warnings
warnings.filterwarnings("ignore")

#### 1.  Use the function defined in acquire.py to load the iris data.

In [67]:
iris = a.get_iris_data()
iris.head()

,species_id,species_name,sepal_length,sepal_width,petal_length,petal_width
0,1,setosa,5.1,3.5,1.4,0.2
1,1,setosa,4.9,3.0,1.4,0.2
2,1,setosa,4.7,3.2,1.3,0.2
3,1,setosa,4.6,3.1,1.5,0.2
4,1,setosa,5.0,3.6,1.4,0.2


#### 2.  Drop the species_id and measurement_id columns.

In [ ]:
iris = iris.drop(['species_id'], axis = 1)

#### 3.  Rename the species_name column to just species.

In [ ]:
#rename
iris = iris.rename(columns={'species_name':'species'})
iris.head

#### 4.  Create dummy variables of the species name.

In [ ]:
#create dummy variables
dummy_df = pd.get_dummies(iris['species'], drop_first=False)
dummy_df.head()

In [ ]:
#join with original dataframe
iris = pd.concat([iris, dummy_df], axis=1)
iris.head()

##### 5.  Create a function named prep_iris that accepts the untransformed iris data, and returns the data with the transformations above applied.

In [71]:
def prep_iris(df):
    '''
    This function drops the speicies_id column, renames the species_name
    column and creates dummy variables for the species and merges this with
    original dataframe.
    '''
    # Drop species id column
    df = df.drop(['species_id'], axis = 1)
    
    #rename species_name to species
    df = df.rename(columns={'species_name':'species'})
    
    #create dummy variables
    dummy_df = pd.get_dummies(df['species'], drop_first=False)
    
    #join with original dataframe
    df = pd.concat([df, dummy_df], axis=1)
    
    return df

In [72]:
prep_iris(iris).head()

,species,sepal_length,sepal_width,petal_length,petal_width,setosa,versicolor,virginica
0,setosa,5.1,3.5,1.4,0.2,1,0,0
1,setosa,4.9,3.0,1.4,0.2,1,0,0
2,setosa,4.7,3.2,1.3,0.2,1,0,0
3,setosa,4.6,3.1,1.5,0.2,1,0,0
4,setosa,5.0,3.6,1.4,0.2,1,0,0


In [ ]:
prep_iris(df).head()